# Introduction

This notebook utilises MMA sports data from a Kaggle dataset. The aim of this notebook was too practice my data cleaning and preparation skills whilst also answering some rudimentry questions I personally had about the UFC and MMA in general. 

### Import packages

In [39]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
pd.options.display.max_columns = 200

### Load data

In [40]:
# Import datasets
events = pd.read_csv('data/ufc_event_data.csv')
fights = pd.read_csv('data/ufc_fight_data.csv')
fight_stats = pd.read_csv('data/ufc_fight_stat_data.csv')
fighters = pd.read_csv('data/ufc_fighter_data.csv')

# Data Cleaning and Preparation

The way that I approached preparing the data was to first, get a quick look through all tables to get an idea of relevant columns, data types to set, missing values. During this process I'm making a mental note of some joins/merges I will likely make.  

### Table 1: `events`
- Convert column to datetime
- Drop unnecessary columns

In [41]:
print(events.shape)
print(events.info())
display(events.head())

(665, 7)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 665 entries, 0 to 664
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   event_id       665 non-null    int64 
 1   event_name     665 non-null    object
 2   event_date     665 non-null    object
 3   event_city     665 non-null    object
 4   event_state    616 non-null    object
 5   event_country  665 non-null    object
 6   event_url      665 non-null    object
dtypes: int64(1), object(6)
memory usage: 36.5+ KB
None


,event_id,event_name,event_date,event_city,event_state,event_country,event_url
0,665,UFC Fight Night: Dawson vs. Green,2023-10-07,Las Vegas,Nevada,USA,http://ufcstats.com/event-details/c8a49ff2acb6...
1,664,UFC Fight Night: Fiziev vs. Gamrot,2023-09-23,Las Vegas,Nevada,USA,http://ufcstats.com/event-details/c945adc22c2b...
2,663,UFC Fight Night: Grasso vs. Shevchenko 2,2023-09-16,Las Vegas,Nevada,USA,http://ufcstats.com/event-details/8fa2b0657236...
3,662,UFC 293: Adesanya vs. Strickland,2023-09-09,Sydney,New South Wales,Australia,http://ufcstats.com/event-details/ece280745f87...
4,661,UFC Fight Night: Gane vs. Spivac,2023-09-02,Paris,Ile-de-France,France,http://ufcstats.com/event-details/ef61d9f5176b...


In [42]:
# Convert event_date to datetime and extract year
events['event_date'] = pd.to_datetime(events['event_date'])

# Keep only needed columns
events = events[['event_id', # 'event_name', 
                 'event_date', # 'event_city', 'event_state',
                 # 'event_country', 'event_url'
]].copy()

print(events.shape)
print(events.info())

(665, 2)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 665 entries, 0 to 664
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   event_id    665 non-null    int64         
 1   event_date  665 non-null    datetime64[ns]
dtypes: datetime64[ns](1), int64(1)
memory usage: 10.5 KB
None


Export dataframe as feather (to preserve datatypes)

In [43]:
events.to_feather('data/events_clean.feather')

### Table 2: `fights`
- Drop unneccesary columns
- Deal with missing values
- Create column for total fight duration

In [44]:
print(fights.shape)
print(fights.info())
display(fights.head())

(7218, 15)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7218 entries, 0 to 7217
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   fight_id        7218 non-null   int64  
 1   event_id        7218 non-null   int64  
 2   referee         7186 non-null   object 
 3   f_1             7199 non-null   float64
 4   f_2             7205 non-null   float64
 5   winner          7203 non-null   float64
 6   num_rounds      7218 non-null   object 
 7   title_fight     7218 non-null   object 
 8   weight_class    7205 non-null   object 
 9   gender          7218 non-null   object 
 10  result          7218 non-null   object 
 11  result_details  7201 non-null   object 
 12  finish_round    7218 non-null   int64  
 13  finish_time     7218 non-null   object 
 14  fight_url       7218 non-null   object 
dtypes: float64(3), int64(3), object(9)
memory usage: 846.0+ KB
None


,fight_id,event_id,referee,f_1,f_2,winner,num_rounds,title_fight,weight_class,gender,result,result_details,finish_round,finish_time,fight_url
0,7218,664,Herb Dean,2976.0,2884.0,2884.0,5,F,Lightweight,M,KO/TKO,to \n Leg Injury,2,2:03,http://ufcstats.com/fight-details/23a604f46028...
1,7217,664,Mark Smith,1662.0,2464.0,1662.0,3,F,Featherweight,M,Decision,Unanimous,3,5:00,http://ufcstats.com/fight-details/da1b37edb8cc...
2,7216,664,Kerry Hatley,981.0,179.0,981.0,3,F,Women's Strawweight,F,KO/TKO,Punches to Head From Mount,2,2:42,http://ufcstats.com/fight-details/d8335b728604...
3,7215,664,Dan Miragliotta,3831.0,2974.0,3831.0,3,F,Welterweight,M,Submission,Rear Naked Choke,2,4:32,http://ufcstats.com/fight-details/bf647be41de3...
4,7214,664,Herb Dean,1108.0,2320.0,2320.0,3,F,Featherweight,M,Submission,Guillotine Choke From Bottom Guard,1,3:12,http://ufcstats.com/fight-details/6e1bf1b163b3...


In [45]:
# Remove unneeded columns
fights = fights[[
    'fight_id', 'event_id', # 'referee', 
    'f_1', 'f_2', 'winner', 'num_rounds',
    'title_fight', 'weight_class', 'gender', 'result', # 'result_details',
    'finish_round', 'finish_time' #, 'fight_url'
]].copy()

Examining missing values for the fights table, led me to question if the missing values for the `winner` column, which contains the fighter_id of the winning fighter, was due to cases where there was no winner (e.g. No Contest). I indexed for rows with missing `winner` values and had a look at the `result` column, which revealed that this wasn't the case at all.

In [46]:
fights.isnull().sum()

fight_id         0
event_id         0
f_1             19
f_2             13
winner          15
num_rounds       0
title_fight      0
weight_class    13
gender           0
result           0
finish_round     0
finish_time      0
dtype: int64

In [47]:
# Investigate to see if a null winner value is due to a no contest
fights[fights['winner'].isnull()].head()

,fight_id,event_id,f_1,f_2,winner,num_rounds,title_fight,weight_class,gender,result,finish_round,finish_time
9,7209,664,NaN,2808.0,NaN,3,F,Women's Strawweight,F,Decision,3,5:00
547,6671,619,3966.0,NaN,NaN,3,F,Bantamweight,M,Decision,3,5:00
571,6647,617,NaN,1241.0,NaN,3,F,Bantamweight,M,Decision,3,5:00
686,6532,607,NaN,2871.0,NaN,3,F,Lightweight,M,KO/TKO,1,1:14
759,6459,601,NaN,3095.0,NaN,3,F,Bantamweight,M,KO/TKO,1,2:48


Dropping rows with missing values, if a column's missing value count is below a threshold of 5% of column length.

In [48]:
# Find the five percent threshold
threshold = len(fights) * 0.05

# Create a filter
cols_to_drop = fights.columns[fights.isna().sum() <= threshold]

# Drop missing values for columns below the threshold
fights.dropna(subset=cols_to_drop, inplace=True)
print(fights.isna().sum()) 

fight_id        0
event_id        0
f_1             0
f_2             0
winner          0
num_rounds      0
title_fight     0
weight_class    0
gender          0
result          0
finish_round    0
finish_time     0
dtype: int64


In [49]:
# Create a column for fight duration in seconds
fights[['final_min', 'final_sec']] = fights['finish_time'].str.split(pat=':', expand=True)
# convert to minute and seconds to timedeltas and sum
fights['final_min'] = fights['final_min'].astype(int)
fights['final_sec'] = fights['final_sec'].astype(int)
fights['final_round_duration'] = pd.to_timedelta(fights['final_min'], unit='m') \
    + pd.to_timedelta(fights['final_sec'], unit='s')
# multiply (number of rounds - 1) by 5 (5 minutes per round) and sum with final round duration for total fight duration
fights['fight_duration'] = pd.to_timedelta(
    (fights['finish_round'] - 1) * 5,
    unit='m'
) + fights['final_round_duration']
# Create a column for total fight duration in seconds
fights['fight_duration_sec'] = fights['fight_duration'].dt.total_seconds() \
    .astype('int')

Convert data types

In [50]:
fights['num_rounds'].value_counts()

num_rounds
3    6350
5     627
1     157
N      28
2      11
Name: count, dtype: int64

In [51]:
fights = fights.query('num_rounds != "N"').copy()
# Convert datatypes
fights[["f_1", "f_2", "winner", "num_rounds"]] = fights[
    ["f_1", "f_2", "winner", "num_rounds"]
].astype("int")

Drop unnecessary columns

In [52]:
fights.columns

Index(['fight_id', 'event_id', 'f_1', 'f_2', 'winner', 'num_rounds',
       'title_fight', 'weight_class', 'gender', 'result', 'finish_round',
       'finish_time', 'final_min', 'final_sec', 'final_round_duration',
       'fight_duration', 'fight_duration_sec'],
      dtype='object')

In [53]:
fights = fights[
    ['fight_id', 'event_id', 'f_1', 'f_2', 'winner', 'num_rounds',
       'title_fight', 'weight_class', 'gender', 'result', # 'finish_round',
       # 'finish_time', 'final_min', 'final_sec', 'final_round_duration',
       # 'fight_duration', 
       'fight_duration_sec']
]

In [54]:
fights.loc[
    fights['weight_class'].str.contains('Strawweight'),
    'weight_class_lbs'
] = 115
fights.loc[
    fights['weight_class'].str.contains('Flyweight'),
    'weight_class_lbs'
] = 125
fights.loc[
    fights['weight_class'].str.contains('Bantamweight'),
    'weight_class_lbs'
] = 135
fights.loc[
    fights['weight_class'].str.contains('Featherweight'),
    'weight_class_lbs'
] = 145
fights.loc[
    fights['weight_class'].str.contains('Lightweight'),
    'weight_class_lbs'
] = 155
fights.loc[
    fights['weight_class'].str.contains('Welterweight'),
    'weight_class_lbs'
] = 170
fights.loc[
    fights['weight_class'].str.contains('Middleweight'),
    'weight_class_lbs'
] = 185
fights.loc[
    fights['weight_class'].str.contains('Light Heavyweight'),
    'weight_class_lbs'
] = 205
fights.loc[
    fights['weight_class'].str.contains('Heavyweight'),
    'weight_class_lbs'
] = 265


In [55]:
print(fights.info())

<class 'pandas.core.frame.DataFrame'>
Index: 7145 entries, 0 to 7186
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   fight_id            7145 non-null   int64  
 1   event_id            7145 non-null   int64  
 2   f_1                 7145 non-null   int64  
 3   f_2                 7145 non-null   int64  
 4   winner              7145 non-null   int64  
 5   num_rounds          7145 non-null   int64  
 6   title_fight         7145 non-null   object 
 7   weight_class        7145 non-null   object 
 8   gender              7145 non-null   object 
 9   result              7145 non-null   object 
 10  fight_duration_sec  7145 non-null   int64  
 11  weight_class_lbs    7010 non-null   float64
dtypes: float64(1), int64(7), object(4)
memory usage: 725.7+ KB
None


Export dataframe as feather (to preserve datatypes)

In [56]:
fights.to_feather('data/fights_clean.feather')

### Table : `fight_stats`
- Deal will missing values
- Convert data types
    - Convert string to time data
- Feature engineering

In [57]:
print(fight_stats.shape)
print(fight_stats.info())
display(fight_stats.head())

(14436, 14)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14436 entries, 0 to 14435
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   fight_stat_id       14436 non-null  int64  
 1   fight_id            14436 non-null  int64  
 2   fighter_id          14404 non-null  float64
 3   knockdowns          14394 non-null  float64
 4   total_strikes_att   14394 non-null  float64
 5   total_strikes_succ  14394 non-null  float64
 6   sig_strikes_att     14394 non-null  float64
 7   sig_strikes_succ    14394 non-null  float64
 8   takedown_att        14394 non-null  float64
 9   takedown_succ       14394 non-null  float64
 10  submission_att      14394 non-null  float64
 11  reversals           14394 non-null  float64
 12  ctrl_time           14394 non-null  object 
 13  fight_url           14436 non-null  object 
dtypes: float64(10), int64(2), object(2)
memory usage: 1.5+ MB
None


,fight_stat_id,fight_id,fighter_id,knockdowns,total_strikes_att,total_strikes_succ,sig_strikes_att,sig_strikes_succ,takedown_att,takedown_succ,submission_att,reversals,ctrl_time,fight_url
0,14436,7218,2976.0,0.0,34.0,19.0,32.0,18.0,0.0,0.0,0.0,0.0,0:00,http://ufcstats.com/fight-details/23a604f46028...
1,14435,7218,2884.0,0.0,42.0,17.0,40.0,16.0,6.0,1.0,0.0,0.0,1:28,http://ufcstats.com/fight-details/23a604f46028...
2,14434,7217,1662.0,0.0,59.0,37.0,40.0,23.0,15.0,5.0,1.0,0.0,7:33,http://ufcstats.com/fight-details/da1b37edb8cc...
3,14433,7217,2464.0,0.0,72.0,32.0,55.0,18.0,0.0,0.0,0.0,1.0,2:11,http://ufcstats.com/fight-details/da1b37edb8cc...
4,14432,7216,981.0,0.0,130.0,90.0,102.0,70.0,1.0,1.0,0.0,0.0,2:03,http://ufcstats.com/fight-details/d8335b728604...


`ctrl_time` missing values are set to string '--'. Replace these values with null.

In [58]:
fight_stats['ctrl_time'].mask(fight_stats['ctrl_time']=='--', inplace=True)

Deal with missing values

In [59]:
fight_stats.isna().sum()

fight_stat_id           0
fight_id                0
fighter_id             32
knockdowns             42
total_strikes_att      42
total_strikes_succ     42
sig_strikes_att        42
sig_strikes_succ       42
takedown_att           42
takedown_succ          42
submission_att         42
reversals              42
ctrl_time             398
fight_url               0
dtype: int64

In [60]:
# Find the five percent threshold
threshold = len(fight_stats) * 0.05
print(f'threshold: {threshold}')
# Create a filter
cols_to_drop = fight_stats.columns[fight_stats.isna().sum() <= threshold]

# Drop missing values for columns below the threshold
fight_stats.dropna(subset=cols_to_drop, inplace=True)
print(fight_stats.isna().sum()) 

threshold: 721.8000000000001
fight_stat_id         0
fight_id              0
fighter_id            0
knockdowns            0
total_strikes_att     0
total_strikes_succ    0
sig_strikes_att       0
sig_strikes_succ      0
takedown_att          0
takedown_succ         0
submission_att        0
reversals             0
ctrl_time             0
fight_url             0
dtype: int64


Convert data types

In [61]:
# Convert minute and second columns to timedeltas and sum
fight_stats[['ctrl_time_min', 'ctrl_time_sec']] = fight_stats['ctrl_time'].str.split(':', expand=True)
fight_stats[['ctrl_time_min', 'ctrl_time_sec']] = fight_stats[['ctrl_time_min', 'ctrl_time_sec']].astype('int')

fight_stats['ctrl_time_sec'] = (
    (fight_stats['ctrl_time_min'] * 60) + fight_stats['ctrl_time_sec']
)

fight_stats.drop(columns=['ctrl_time_min', 'ctrl_time', 'fight_url'], inplace=True)

In [62]:
fight_stats[[
    'fighter_id', 'knockdowns',
    'total_strikes_att', 'total_strikes_succ', 'sig_strikes_att',
    'sig_strikes_succ', 'takedown_att', 'takedown_succ', 'submission_att',
    'reversals'
]] = fight_stats[[
    'fighter_id', 'knockdowns',
    'total_strikes_att', 'total_strikes_succ', 'sig_strikes_att',
    'sig_strikes_succ', 'takedown_att', 'takedown_succ', 'submission_att',
    'reversals'
]].astype('int')

fight_stats.info()

<class 'pandas.core.frame.DataFrame'>
Index: 14006 entries, 0 to 14039
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype
---  ------              --------------  -----
 0   fight_stat_id       14006 non-null  int64
 1   fight_id            14006 non-null  int64
 2   fighter_id          14006 non-null  int64
 3   knockdowns          14006 non-null  int64
 4   total_strikes_att   14006 non-null  int64
 5   total_strikes_succ  14006 non-null  int64
 6   sig_strikes_att     14006 non-null  int64
 7   sig_strikes_succ    14006 non-null  int64
 8   takedown_att        14006 non-null  int64
 9   takedown_succ       14006 non-null  int64
 10  submission_att      14006 non-null  int64
 11  reversals           14006 non-null  int64
 12  ctrl_time_sec       14006 non-null  int64
dtypes: int64(13)
memory usage: 1.5 MB


Feature engineering

In [63]:
# Total strike accuracy
fight_stats['total_strike_acc_pct'] = (
    fight_stats['total_strikes_succ'] / fight_stats['total_strikes_att']
) * 100
# Significant strike accuracy
fight_stats['sig_strike_acc_pct'] = (
    fight_stats['sig_strikes_succ'] / fight_stats['sig_strikes_att']
) * 100
# Takedown accuracy
fight_stats['takedown_acc_pct'] = (
    fight_stats['takedown_succ'] / fight_stats['takedown_att']
) * 100

Export dataframe as feather (to preserve datatypes)

In [64]:
fight_stats.to_feather('data/fight_stats_clean.feather')

### Table `fighter`
- Drop unnecessary columns
- Deal with missing values
- Convert data types

In [65]:
print(fighters.shape)
print(fighters.info())
display(fighters.head())

(4107, 14)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4107 entries, 0 to 4106
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   fighter_id          4107 non-null   int64  
 1   fighter_f_name      4107 non-null   object 
 2   fighter_l_name      4092 non-null   object 
 3   fighter_nickname    2250 non-null   object 
 4   fighter_height_cm   3797 non-null   float64
 5   fighter_weight_lbs  4020 non-null   float64
 6   fighter_reach_cm    2166 non-null   float64
 7   fighter_stance      3273 non-null   object 
 8   fighter_dob         3349 non-null   object 
 9   fighter_w           4107 non-null   int64  
 10  fighter_l           4107 non-null   int64  
 11  fighter_d           4107 non-null   int64  
 12  fighter_nc_dq       482 non-null    float64
 13  fighter_url         4107 non-null   object 
dtypes: float64(4), int64(4), object(6)
memory usage: 449.3+ KB
None


,fighter_id,fighter_f_name,fighter_l_name,fighter_nickname,fighter_height_cm,fighter_weight_lbs,fighter_reach_cm,fighter_stance,fighter_dob,fighter_w,fighter_l,fighter_d,fighter_nc_dq,fighter_url
0,4107,Tom,Aaron,NaN,NaN,155.0,NaN,NaN,1978-07-13,5,3,0,NaN,http://ufcstats.com/fighter-details/93fe7332d1...
1,4106,Danny,Abbadi,The Assassin,180.34,155.0,NaN,Orthodox,1983-07-03,4,6,0,NaN,http://ufcstats.com/fighter-details/15df64c02b...
2,4105,Nariman,Abbasov,Bayraktar,172.72,155.0,167.64,Orthodox,1994-02-01,28,4,0,NaN,http://ufcstats.com/fighter-details/59a9d6dac6...
3,4104,David,Abbott,Tank,182.88,265.0,NaN,Switch,NaN,10,15,0,NaN,http://ufcstats.com/fighter-details/b361180739...
4,4103,Hamdy,Abdelwahab,The Hammer,187.96,264.0,182.88,Southpaw,1993-01-22,5,0,0,1.0,http://ufcstats.com/fighter-details/3329d692ae...


Drop unecessary columns

In [66]:
fighters = fighters[[
    'fighter_id', # 'fighter_f_name', 'fighter_l_name', 'fighter_nickname',
       'fighter_height_cm', # 'fighter_weight_lbs', 
       'fighter_reach_cm',
       # 'fighter_stance', 
       'fighter_dob', # 'fighter_w', 'fighter_l', 'fighter_d',
       # 'fighter_nc_dq', 'fighter_url'
]].copy()

Missing values


In [67]:
fighters.isna().sum()

fighter_id              0
fighter_height_cm     310
fighter_reach_cm     1941
fighter_dob           758
dtype: int64

There are no columns with missing values that meet the threshold. Dropping these would lead to losing too much information so will decide to keep.

In [68]:
# Find the five percent threshold
threshold = len(fighters) * 0.05
print(f'threshold: {threshold}')
# Create a filter
cols_to_drop = fighters.columns[fighters.isna().sum() <= threshold]
print(f'threshold: {cols_to_drop}')

threshold: 205.35000000000002
threshold: Index(['fighter_id'], dtype='object')


Convert data types

In [69]:
fighters['fighter_dob'] = pd.to_datetime(fighters['fighter_dob'])

Export dataframe as feather (to preserve datatypes)

In [70]:
fighters.to_feather('data/fighters_clean.feather')

### Joining/merging tables
- Merge
- Append tables (concat)
- Feature engineering

In [71]:
df0 = fight_stats.merge(fighters, how="left", on="fighter_id")

df1 = (
    df0.merge(
        fights,
        how="left",
        left_on=["fight_id", "fighter_id"],
        right_on=["fight_id", "f_1"],
        indicator=True,
    )
    .query('_merge == "both"')
    .drop(columns=["_merge"])
)

df2 = (
    df0.merge(
        fights,
        how="left",
        left_on=["fight_id", "fighter_id"],
        right_on=["fight_id", "f_2"],
        indicator=True,
    )
    .query('_merge == "both"')
    .drop(columns=["_merge"])
)

# Merge to get opposition fighter details
df1 = df1.merge(
    df2[["fight_id", "fighter_height_cm", "fighter_reach_cm", "fighter_dob"]],
    how="left",
    on="fight_id",
    suffixes=("", "_opp"),
)
df2 = df2.merge(
    df1[["fight_id", "fighter_height_cm", "fighter_reach_cm", "fighter_dob"]],
    how="left",
    on="fight_id",
    suffixes=("", "_opp"),
)

ufc_df = pd.concat([df1, df2])
ufc_df = ufc_df.reset_index(drop=True)
ufc_df["fight_date"] = ufc_df.merge(
    events, how="left", on="event_id"
)["event_date"]

Feature engineering

In [72]:
ufc_df['reach_adv_cm'] = ufc_df['fighter_reach_cm'] - ufc_df['fighter_reach_cm_opp']
ufc_df['height_adv_cm'] = ufc_df['fighter_height_cm'] - ufc_df['fighter_height_cm_opp']

ufc_df.loc[ufc_df['fighter_id'] == ufc_df['winner'], 'win'] = 1
ufc_df.loc[ufc_df['fighter_id'] != ufc_df['winner'], 'win'] = 0

ufc_df['fighter_age'] = (
    (
    ufc_df['fight_date'] - ufc_df['fighter_dob']
) / np.timedelta64(365, "D")
).round()

ufc_df['fighter_age_opp'] = (
    (
    ufc_df['fight_date'] - ufc_df['fighter_dob_opp']
) / np.timedelta64(365, "D")
).round()

ufc_df['age_diff'] = ufc_df['fighter_age'] - ufc_df['fighter_age_opp']

# Weight categories

ufc_df.loc[
    ufc_df['weight_class'].str.contains('Strawweight'),
    'weight_class_lbs'
] = 115
ufc_df.loc[
    ufc_df['weight_class'].str.contains('Flyweight'),
    'weight_class_lbs'
] = 125
ufc_df.loc[
    ufc_df['weight_class'].str.contains('Bantamweight'),
    'weight_class_lbs'
] = 135
ufc_df.loc[
    ufc_df['weight_class'].str.contains('Featherweight'),
    'weight_class_lbs'
] = 145
ufc_df.loc[
    ufc_df['weight_class'].str.contains('Lightweight'),
    'weight_class_lbs'
] = 155
ufc_df.loc[
    ufc_df['weight_class'].str.contains('Welterweight'),
    'weight_class_lbs'
] = 170
ufc_df.loc[
    ufc_df['weight_class'].str.contains('Middleweight'),
    'weight_class_lbs'
] = 185
ufc_df.loc[
    ufc_df['weight_class'].str.contains('Light Heavyweight'),
    'weight_class_lbs'
] = 205
ufc_df.loc[
    ufc_df['weight_class'].str.contains('Heavyweight'),
    'weight_class_lbs'
] = 265

# Create weight category column
weight_cat = ['Light', 'Medium', 'Heavy']

weight_light = "Straw|Fly|Bantam|Feather"
weight_medium = "Lightweight|Welter|Middle"
weight_heavy = "Heavy"

conditions = [
    ufc_df['weight_class'].str.contains(weight_light),
    ufc_df['weight_class'].str.contains(weight_medium),
    ufc_df['weight_class'].str.contains(weight_heavy)
]

ufc_df['weight_category'] = np.select(conditions, weight_cat, default="Other")

Drop unnecessary columns

In [73]:
ufc_df.columns

Index(['fight_stat_id', 'fight_id', 'fighter_id', 'knockdowns',
       'total_strikes_att', 'total_strikes_succ', 'sig_strikes_att',
       'sig_strikes_succ', 'takedown_att', 'takedown_succ', 'submission_att',
       'reversals', 'ctrl_time_sec', 'total_strike_acc_pct',
       'sig_strike_acc_pct', 'takedown_acc_pct', 'fighter_height_cm',
       'fighter_reach_cm', 'fighter_dob', 'event_id', 'f_1', 'f_2', 'winner',
       'num_rounds', 'title_fight', 'weight_class', 'gender', 'result',
       'fight_duration_sec', 'weight_class_lbs', 'fighter_height_cm_opp',
       'fighter_reach_cm_opp', 'fighter_dob_opp', 'fight_date', 'reach_adv_cm',
       'height_adv_cm', 'win', 'fighter_age', 'fighter_age_opp', 'age_diff',
       'weight_category'],
      dtype='object')

In [74]:
ufc_df = ufc_df[
    [# 'fight_stat_id', 
     'fight_id', 'fighter_id', 'knockdowns',
       'total_strikes_att', 'total_strikes_succ', 'sig_strikes_att',
       'sig_strikes_succ', 'takedown_att', 'takedown_succ', 'submission_att',
       'reversals', 'ctrl_time_sec', 'total_strike_acc_pct',
       'sig_strike_acc_pct', 'takedown_acc_pct', 'fighter_height_cm',
       'fighter_reach_cm', 'fighter_dob', # 'event_id', 'f_1', 'f_2', 'winner',
       'num_rounds', 'title_fight', 'weight_class', 'weight_class_lbs', 'weight_category', 'gender', 'result',
       'fight_duration_sec', 'fighter_height_cm_opp', 'fighter_reach_cm_opp',
       'fighter_dob_opp', 'fight_date', 'reach_adv_cm', 'height_adv_cm', 'win',
       'fighter_age', 'fighter_age_opp', 'age_diff']
].copy()

Export dataframe as feather (to preserve datatypes)

In [75]:
ufc_df.to_feather('data/ufc_df.feather')